# 使用LSTM的恶意网页识别

**作者:** [深渊上的坑](https://github.com/edencfc) <br>
**日期:** 2021.05 <br>
**摘要:** 本示例教程介绍如何使用飞桨完成一个恶意网页分类任务。通过使用飞桨搭建LSTM网络，组件一个网页内容分类的模型，并在示例的数据集上完成恶意网页的识别。

## 一、环境配置

本教程基于Paddle 2.0 编写，如果你的环境不是本版本，请先参考官网[安装](https://www.paddlepaddle.org.cn/install/quick) Paddle 2.0 。

In [2]:
!pip install lxml -i https://mirror.baidu.com/pypi/simple/
!pip install bs4 -i https://mirror.baidu.com/pypi/simple/
!pip install html5lib -i https://mirror.baidu.com/pypi/simple/

In [2]:
import os
import sys
import codecs
import chardet
import shutil
import time
import numpy as np
import pandas as pd
import jieba
from tqdm import tqdm, trange
from bs4 import BeautifulSoup
from functools import partial
import paddle
import paddlenlp
import paddle.nn as nn
import paddle.nn.functional as F
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.datasets import MapDatasetWrapper

print(paddle.__version__)

2.0.2


## 二、数据加载

### 2.1 数据集下载

将使用 [https://www.heywhale.com/](https://cdn.kesci.com/%E6%81%B6%E6%84%8F%E7%BD%91%E9%A1%B5%E5%88%86%E6%9E%90.zip) 提供的恶意网页分析样本作为数据集，来完成本任务。该数据集含有169个恶意网页。

In [3]:
!wget -c https://cdn.kesci.com/%E6%81%B6%E6%84%8F%E7%BD%91%E9%A1%B5%E5%88%86%E6%9E%90.zip && mv *.zip MaliciousWebpage.zip && unzip MaliciousWebpage.zip 

### 2.2 读取文件信息

读取文件列表信息。

In [ ]:
columns = ['id', 'flag', 'filename', 'url']
tempdf = pd.read_csv('MaliciousWebpage/file_list.txt', sep=',',skiprows=0, header=None, names=columns, skipfooter=0)

### 2.3 使用下采样处理数据集不均衡问题

在数据集中，正常网页样本有9700个，而恶意网页样本近169个，数据集严重不均衡，使用下采样的方法，随机筛选出500个正常网页的样本参与训练。

In [ ]:
n_page = tempdf[tempdf['flag']=='n']
# 对正常页面进行随机采样
n_page = n_page.sample(n=500)
# 提取全部被黑页面样本
d_page = tempdf[tempdf['flag']=='d']
# 合并样本
train_page = pd.concat([n_page,d_page],axis=0)
# 合并样本
train_page = pd.concat([n_page,d_page],axis=0)
# 做一个乱序
train_page = train_page.sample(frac = 1) 

### 2.4 进行字符集编码处理

解析数据集中网页内容时，可能出现因字符集编码不一致导致的读取错误，因此，要先对进行批量字符集编码转换，对数据集进行清洗。

In [ ]:
!mkdir TrainWebpage && mkdir TrainWebpage/file1

In [ ]:
for filename in tqdm(train_page['filename']):
    # 这里要先做个判断，有file_list里面的文件不存在
    if os.path.exists('MaliciousWebpage/file1/'+filename):
        # 读取文件，获取字符集
        content = codecs.open('MaliciousWebpage/file1/'+filename,'rb').read()
        source_encoding = chardet.detect(content)['encoding']
        # 个别文件的source_encoding是None，这里要先进行筛选
        if source_encoding is None:
            pass
        # 只对字符集是gb2312格式的文件尝试转码
        elif source_encoding == 'gb2312':
            # 转码如果失败，就跳过该文件
            try:
                content = content.decode(source_encoding).encode('utf-8')
                codecs.open('TrainWebpage/file1/'+filename,'wb').write(content)
            except UnicodeDecodeError:
                print(filename + "读取失败")
                pass
        # 字符集是utf-8格式的文件直接保存
        elif source_encoding == 'utf-8':
            codecs.open('TrainWebpage/file1/'+filename,'wb').write(content)
        else:
            pass
    else:
        pass

### 2.5 提取网页内容，划分训练集、验证集、测试集

被黑网页的一个典型特征是恶意插入的内容大量集中在HTML页面底部，因此可以提取网页末尾的HTML内容作为输入LSTM的文本信息。

In [ ]:
for i, filename in enumerate(train_page['filename']):
    # 这里要先做个判断，有file_list里面的文件不存在
    if os.path.exists('TrainWebpage/file1/'+filename):
        # 读取文件，解析HTML页面
        html = BeautifulSoup(open('TrainWebpage/file1/'+filename),'html.parser', from_encoding='utf-8')
        text = ''.join(list(html.stripped_strings)[-20:])
        # 去掉多余的换行符（部分数据最后解析结果为）
        text = text.replace("\n", "")
        text = text.replace(" ", ",")
        # real_label = train_page['flag'][train_page['filename']==filename].values[0]
        if i % 5 == 0:
            if train_page['flag'][train_page['filename']==filename].values[0] == 'n':
                with open("webtest.txt","a+") as f:
                    f.write(text[-100:] + '\t' + '0' + '\n')
            elif train_page['flag'][train_page['filename']==filename].values[0] == 'd':
                with open("webtest.txt","a+") as f:
                    f.write(text[-100:] + '\t' + '1' + '\n')
        elif i % 5 == 1:
            if train_page['flag'][train_page['filename']==filename].values[0] == 'n':
                with open("webdev.txt","a+") as f:
                    f.write(text[-100:] + '\t' + '0' + '\n')
            elif train_page['flag'][train_page['filename']==filename].values[0] == 'd':
                with open("webdev.txt","a+") as f:
                    f.write(text[-100:] + '\t' + '1' + '\n')
        else:
            if train_page['flag'][train_page['filename']==filename].values[0] == 'n':
                with open("webtrain.txt","a+") as f:
                    f.write(text[-100:] + '\t' + '0' + '\n')
            elif train_page['flag'][train_page['filename']==filename].values[0] == 'd':
                with open("webtrain.txt","a+") as f:
                    f.write(text[-100:] + '\t' + '1' + '\n')
    else:
        pass

### 2.6 自定义数据集

In [4]:
class SelfDefinedDataset(paddle.io.Dataset):
    def __init__(self, data):
        super(SelfDefinedDataset, self).__init__()
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)
        
    def get_labels(self):
        return ["0", "1"]

def txt_to_list(file_name):
    res_list = []
    for line in open(file_name):
        res_list.append(line.strip().split('\t'))
    return res_list

trainlst = txt_to_list('webtrain.txt')
devlst = txt_to_list('webdev.txt')
testlst = txt_to_list('webtest.txt')

train_ds, dev_ds, test_ds = SelfDefinedDataset.get_datasets([trainlst, devlst, testlst])

In [5]:
# 准备标签
label_list = train_ds.get_labels()
print(label_list)
# 查看样本
for i in range(5):
    print (train_ds[i])

['0', '1']
['年以内,2万公里以内SUV1年以内易车二手车体验更好，速度更快立即前往APP看电脑版看微信版提意见购车热线：4000-189-167(,9:00,–,21:00,)易车二手车,m.taoche.com', '0']
['ipaime.com/thread-694853-1-1.htmlcoryphaei.com/forum.php?mod=viewthread&tid=3074054回复返回版块参与回复©,栖霞商业网', '0']
['大直街店集体课表人和国际健身俱乐部首页集体课表联系我们扫描二维码用手机访问本站由业界领先的搜狐快站免费提供技术支持人和国际健身俱乐部人和健身大直街店集体课表15小时前阅读Powered,by,搜狐快站', '0']
['个人帐户工作或学校帐户单位或学校未分配帐户?使用,Microsoft,帐户登录厌烦了这个帐户名称?重命名您的个人,Microsoft,帐户。©,2017,Microsoft使用条款隐私与,Cookie', '0']
['ONGAB4yONGAB4y精绝美女-在线直播在线播放-高清无水印九狮赌城-美女荷官All,rights,reserved.Copyright,©2016,&2017', '0']


### 2.7 创建词表

接下来创建中文的词表，词表的内容来自对训练集文本的切词。这份词表会用来将英文和中文的句子转换为词的ID构成的序列。词表中还加入了如下两个特殊的词：
- `<pad>`: 用来对较短的句子进行填充。
- `<unk>`: 表示未在词表中出现的词。


In [6]:
dict_path = 'webdict.txt'

#创建数据字典，存放位置：webdict.txt。在生成之前先清空webdict.txt
#在生成all_data.txt之前，首先将其清空
with open(dict_path, 'w') as f:
    f.seek(0)
    f.truncate() 


dict_set = set()
train_data = open('webtrain.txt')
for data in train_data:
    seg = jieba.lcut(data[:-3])
    for datas in seg:
        if not datas is " ":
            dict_set.add(datas)

dicts = open(dict_path,'w')
dicts.write('[PAD]\n')
dicts.write('[UNK]\n')
for data in dict_set:
    dicts.write(data + '\n')
dicts.close()

Building prefix dict from the default dictionary ...
2021-05-13 17:36:40,318 - DEBUG - Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2021-05-13 17:36:40,321 - DEBUG - Loading model from cache /tmp/jieba.cache
Loading model cost 0.832 seconds.
2021-05-13 17:36:41,152 - DEBUG - Loading model cost 0.832 seconds.
Prefix dict has been built successfully.
2021-05-13 17:36:41,154 - DEBUG - Prefix dict has been built successfully.


In [8]:
# 加载词表
def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = {}
    with open(vocab_file, "r", encoding="utf-8") as reader:
        tokens = reader.readlines()
    for index, token in enumerate(tokens):
        token = token.rstrip("\n").split("\t")[0]
        vocab[token] = index
    return vocab
    
vocab = load_vocab('./webdict.txt')

for k, v in vocab.items():
    print(k, v)
    break

[PAD] 0


## 三、网络搭建

### 3.1 构造DataLoader

下面的`create_data_loader`函数用于创建运行和预测时所需要的`DataLoader`对象。

- `paddle.io.DataLoader`返回一个迭代器，该迭代器根据`batch_sampler`指定的顺序迭代返回dataset数据。异步加载数据。
    
- `batch_sampler`：DataLoader通过 batch\_sampler 产生的mini-batch索引列表来 dataset 中索引样本并组成mini-batch
    
- `collate_fn`：指定如何将样本列表组合为mini-batch数据。传给它参数需要是一个callable对象，需要实现对组建的batch的处理逻辑，并返回每个batch的数据。在这里传入的是`prepare_input`函数，对产生的数据进行pad操作，并返回实际长度等。

In [9]:
def convert_example(example, vocab, unk_token_id=1, is_test=False):
    """
    jieba 分词，转换id
    """

    input_ids = []
    for token in jieba.cut(example[0]):
        token_id = vocab.get(token, unk_token_id)
        input_ids.append(token_id)
    valid_length = np.array(len(input_ids), dtype='int64')

    if not is_test:
        label = np.array(example[-1], dtype="int64")
        return input_ids, valid_length, label
    else:
        return input_ids, valid_length


def convert_tokens_to_ids(tokens, vocab):
    """ Converts a token id (or a sequence of id) in a token string
        (or a sequence of tokens), using the vocabulary.
    """

    ids = []
    unk_id = vocab.get('[UNK]', None)
    for token in tokens:
        wid = vocab.get(token, unk_id)
        if wid:
            ids.append(wid)
    return ids

In [10]:
# Reads data and generates mini-batches.
def create_dataloader(dataset,
                      trans_function=None,
                      mode='train',
                      batch_size=1,
                      pad_token_id=0,
                      batchify_fn=None):
    if trans_function:
        dataset = dataset.apply(trans_function, lazy=True)

    # return_list 数据是否以list形式返回
    # collate_fn  指定如何将样本列表组合为mini-batch数据。传给它参数需要是一个callable对象，需要实现对组建的batch的处理逻辑，并返回每个batch的数据。在这里传入的是`prepare_input`函数，对产生的数据进行pad操作，并返回实际长度等。
    dataloader = paddle.io.DataLoader(
        dataset,
        return_list=True,
        batch_size=batch_size,
        collate_fn=batchify_fn)
        
    return dataloader

# python中的偏函数partial，把一个函数的某些参数固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。
trans_function = partial(
    convert_example,
    vocab=vocab,
    unk_token_id=vocab.get('[UNK]', 1),
    is_test=False)

# 将读入的数据batch化处理，便于模型batch化运算。
# batch中的每个句子将会padding到这个batch中的文本最大长度batch_max_seq_len。
# 当文本长度大于batch_max_seq时，将会截断到batch_max_seq_len；当文本长度小于batch_max_seq时，将会padding补齐到batch_max_seq_len.
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=vocab['[PAD]']),  # input_ids
    Stack(dtype="int64"),  # seq len
    Stack(dtype="int64")  # label
): [data for data in fn(samples)]


train_loader = create_dataloader(
    train_ds,
    trans_function=trans_function,
    batch_size=32,
    mode='train',
    batchify_fn=batchify_fn)
dev_loader = create_dataloader(
    dev_ds,
    trans_function=trans_function,
    batch_size=32,
    mode='validation',
    batchify_fn=batchify_fn)
test_loader = create_dataloader(
    test_ds,
    trans_function=trans_function,
    batch_size=32,
    mode='test',
    batchify_fn=batchify_fn)

### 3.2 Encoder部分

使用`LSTMencoder`搭建一个BiLSTM模型用于进行句子建模，得到句子的向量表示。

然后接一个线性变换层，完成二分类任务。

- `paddle.nn.Embedding`组建word-embedding层
- `ppnlp.seq2vec.LSTMEncoder`组建句子建模层
- `paddle.nn.Linear`构造二分类器

In [11]:
class LSTMModel(nn.Layer):
    def __init__(self,
                 vocab_size,
                 num_classes,
                 emb_dim=64,
                 padding_idx=0,
                 lstm_hidden_size=96,
                 direction='forward',
                 lstm_layers=2,
                 dropout_rate=0,
                 pooling_type=None,
                 fc_hidden_size=48):
        super().__init__()

        # 首先将输入word id 查表后映射成 word embedding
        self.embedder = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
            padding_idx=padding_idx)

        # 将word embedding经过LSTMEncoder变换到文本语义表征空间中
        self.lstm_encoder = ppnlp.seq2vec.LSTMEncoder(
            emb_dim,
            lstm_hidden_size,
            num_layers=lstm_layers,
            direction=direction,
            dropout=dropout_rate,
            pooling_type=pooling_type)

        # LSTMEncoder.get_output_dim()方法可以获取经过encoder之后的文本表示hidden_size
        self.fc = nn.Linear(self.lstm_encoder.get_output_dim(), fc_hidden_size)

        # 最后的分类器
        self.output_layer = nn.Linear(fc_hidden_size, num_classes)

    def forward(self, text, seq_len):
        # text shape: (batch_size, num_tokens)
        # print('input :', text.shape)
        
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)
        # print('after word-embeding:', embedded_text.shape)

        # Shape: (batch_size, num_tokens, num_directions*lstm_hidden_size)
        # num_directions = 2 if direction is 'bidirectional' else 1
        text_repr = self.lstm_encoder(embedded_text, sequence_length=seq_len)
        # print('after lstm:', text_repr.shape)


        # Shape: (batch_size, fc_hidden_size)
        fc_out = paddle.tanh(self.fc(text_repr))
        # print('after Linear classifier:', fc_out.shape)

        # Shape: (batch_size, num_classes)
        logits = self.output_layer(fc_out)
        # print('output:', logits.shape)
        
        # probs 分类概率值
        probs = F.softmax(logits, axis=-1)
        # print('output probability:', probs.shape)
        return probs

model= LSTMModel(
        len(vocab),
        len(label_list),
        direction='bidirectional',
        padding_idx=vocab['[PAD]'])
model = paddle.Model(model)

## 四、训练模型

接下来开始训练模型。

- 在训练过程中，使用VisualDL记录训练过程并进行可视化

In [15]:
optimizer = paddle.optimizer.Adam(
        parameters=model.parameters(), learning_rate=1e-4)

loss = paddle.nn.CrossEntropyLoss()
metric = paddle.metric.Accuracy()

model.prepare(optimizer, loss, metric)

In [16]:
# 设置visualdl路径
log_dir = './visualdl'
callback = paddle.callbacks.VisualDL(log_dir=log_dir)

In [21]:
model.fit(train_loader, dev_loader, epochs=100, save_dir='./checkpoints', save_freq=5, callbacks=callback)

In [20]:
# 查看模型在验证集上的表现
results = model.evaluate(dev_loader)
print("Finally val acc: %.5f" % results['acc'])

Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 4/4 - loss: 0.5094 - acc: 0.8426 - 24ms/step
Eval samples: 108
Finally val acc: 0.84259


## 五、预测效果

根据你所使用的计算设备的不同，上面的训练过程可能需要不等的时间。

完成上面的模型训练之后，可以得到一个能够通过网页内容识别恶意网页的模型。接下来查看模型在测试集上的泛化能力。

In [22]:
label_map = {0: '正常页面', 1: '被黑页面'}
results = model.predict(test_loader, batch_size=128)[0]
predictions = []

for batch_probs in results:
    # 映射分类label
    idx = np.argmax(batch_probs, axis=-1)
    idx = idx.tolist()
    labels = [label_map[i] for i in idx]
    predictions.extend(labels)

# 看看预测数据前5个样例分类结果
for idx, data in enumerate(test_ds.data[:5]):
   print('Data: {} \t Label: {}'.format(data[0], predictions[idx]))

Predict begin...
step 4/4 [==============================] - ETA: 0s - 33ms/ste - 21ms/step          
Predict samples: 107
Data: 机超声波保护膜搜狗推广校园推广标签剥离机塑壳断路器厦门婚纱摄影化妆品厂家酿酒设备台湾商务签证行李寄存西安酒店招聘深圳Vi设计不锈钢储罐长沙公司注册湘ICP备15010068号-1,　技术支持：湖南竞网 	 Label: 正常页面
Data: 如果喜欢这个话题，请点击右上角图标分享正在加载...正在加载... 	 Label: 正常页面
Data: 43,站长邮箱:450376843@qq.com免责声明本站的文章和资源来自互联网或者站长的原创,如果有侵犯版权的资源请尽快联系站长,我们会在24h内删除有争议的资源.鄂ICP备15007646号-2 	 Label: 正常页面
Data: 9%的喜欢综艺娱乐的人都会关注这个公众号长按屏幕——识别图中二维码亲，您需要在App中购买后畅听哦~（iPhone新版即将上线，敬请期待）1.,点击右上角,“···”2.,选择,“在浏览器中打开”登录 	 Label: 正常页面
Data: 网娱乐城客服热线首页官方线上游戏的最佳选择澳门赌场娱乐城澳门官方直营新华娱乐城www.5599076.com钱柜娱乐城线上娱乐平台?皇冠现金网下载地址哪个娱乐网站好送68元,www.933jj.com 	 Label: 被黑页面


## The End

你还可以通过变换网络结构，调整数据集，使用预训练模型，尝试不同的参数的方式来进一步提升本示例当中被黑网页识别的效果。同时，也可以尝试在其他的类似的任务中用飞桨来完成实际的实践。